# PyVG: Data Science to predict Video Games sales
>Equipe: Alexis Terrasse, Henri-François Mole, Hsan Drissi, Stephane Lelievre
>
>Promo: DS_Oct21
---
## Scraping VGChartz - Alexis

<img src='https://www.guinnessworldrecords.com/Images/vgchartz-760x444_tcm25-607263.jpg?ezimgfmt=rs:412x241/rscb1/ng:webp/ngcb1' width=500></img>


### Importation des bibliothèques

In [ ]:
from bs4 import BeautifulSoup # Importation de BeautifulSoup
import requests # Importation de import requests 
from time import sleep # Importation de sleep
import pandas as pd # Importation de pandas sous l'alias pd
import lxml.html as lh # Importation de lh 
import unidecode # Importation de unidecode
from tqdm import tqdm # Importation de tqdm

### Définition de fonctions

In [ ]:
# Initialisation des genres
genres=['Action','Action-Adventure','Adventure','Board+Game',
        'Education','Fighting','Misc','MMO','Music','Party','Platform',
        'Puzzle','Racing','Role-Playing','Sandbox','Shooter','Simulation',
        'Sports','Strategy','Visual+Novel']

# Initialisation des parties de l'url de recherche
url_1= 'https://www.vgchartz.com/games/games.php?page='
url_2= '&results=10000&genre='
url_3= '&order=TotalSales&ownership=Both&showtotalsales=1&shownasales=0&showpalsales=0&showjapansales=0&showothersales=0&showpublisher=0&showdeveloper=0&showreleasedate=0&showlastupdate=0&showvgchartzscore=0&showcriticscore=0&showuserscore=0&showshipped=0'


######################################################
######## Fonction des tags des plateformes  ##########
######################################################

def find_console_tags(soup):
    # Les plateforme sont affichées sous forme d'image on récupére l'alt correspondant pour avoir le nom
    consoles = list()
    for img in soup.find_all('img'):
        if 'images/consoles'in img['src']:
            console_tag = (img['src'][17:-6])
            consoles.append(img['alt'])
    return consoles


###############################################
######## Fonction des noms des jeux  ##########
###############################################

def find_names_column(table_path):
    # On récupére le nom du jeu depuis le lien
    names_list = list()
    for row in table_path.xpath('.//tr'):
        for td in row.xpath('.//td'):
            if not td.find('a') is None:
                names_list.append(td.find('a').text.strip()) 
    return names_list


In [ ]:
# On initialise un dataframe vide
df= pd.DataFrame()

# On fait une recherche pour chaque genre
for genre in tqdm(genres):
    url= url_1+url_2+genre+url_3
    
    headermap = {"User-Agent": "Mac Firefox"};
    response = requests.get(url, headers=headermap, stream= True)
    
    #On stocke le contenu du site dans un doc
    page= response.text
    soup = BeautifulSoup(page, "lxml")
    doc = lh.fromstring(response.content)    
    target_table_path = doc.xpath('//*[@id="generalBody"]/table')[0]
    
    # On récupére les infos des colonnes qui ne sont pas correctement scrapées
    names_list = find_names_column(target_table_path)
    consoles = find_console_tags(soup)
    
    sales=[]
    g_game=[]

    for counter,row in enumerate(target_table_path.xpath(".//tr")):
        if counter > 2: 
            i=0
            for td in row.xpath(".//td"):
                if i== 4:
                    sales.append(td.text)
                    g_game.append(genre)
                i+=1

    df_genre= pd.DataFrame(zip(names_list,consoles,g_game,sales), columns=['Name', 'Platform', 'Genre', 'Global_Sales'])

    df= pd.concat([df,df_genre])

In [ ]:
# On réinitialise l'index de df
df= df.reset_index(drop='True')

# On sauvegarde le dataframe dans un csv
df.to_csv('vgchartz.csv', sep=",", encoding='utf-8', index=False)